In [3]:
from dotenv import load_dotenv
load_dotenv()
import os
# from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec
from sentence_transformers import SentenceTransformer

c:\Users\chris\.conda\envs\rag2\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [4]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag",
    dimension=1024,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': 'f7c2f14f1de5eb1892e5e87050094076', 'Date': 'Sat, 24 Aug 2024 14:38:39 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [5]:
import json
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. Emily Johnson',
  'subject': 'Biology',
  'stars': 4,
  'review': "Dr. Johnson's lectures are engaging and informative. She's always willing to help students outside of class."},
 {'professor': 'Prof. Michael Chen',
  'subject': 'Computer Science',
  'stars': 5,
  'review': 'Brilliant instructor! Prof. Chen makes complex algorithms easy to understand. His projects are challenging but rewarding.'},
 {'professor': 'Dr. Sarah Thompson',
  'subject': 'Psychology',
  'stars': 3,
  'review': 'Dr. Thompson knows her subject well, but her lectures can be a bit dry. More interactive sessions would be helpful.'},
 {'professor': 'Prof. David Martinez',
  'subject': 'History',
  'stars': 4,
  'review': 'Prof. Martinez brings history to life with his storytelling. His exams are tough but fair.'},
 {'professor': 'Dr. Rachel Kim',
  'subject': 'Chemistry',
  'stars': 5,
  'review': 'Dr. Kim is an exceptional teacher. Her lab sessions are well-organized and she explains concepts cl

In [6]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CUDA not available")
print(torch.__version__)
print(torch.version.cuda)
torch.cuda.empty_cache()

True
NVIDIA GeForce RTX 2080 SUPER
2.4.0
12.4


In [7]:
processed_data = []
model = SentenceTransformer("dunzhang/stella_en_1.5B_v5", trust_remote_code=True).cuda()

for review in data['reviews']:
    embedding = model.encode([review])
    processed_data.append({
        "values": embedding[0],
        "id": review['professor'],
        "metadata": {
            "review": review['review'],
            "subjects": review['subject'],
            "stars": review['stars'],
        }
    })

c:\Users\chris\.conda\envs\rag2\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
C:\Users\chris\.cache\huggingface\modules\transformers_modules\dunzhang\stella_en_1.5B_v5\d03be74b361d4eb24f42a2fe5bd2e29917df4604\modeling_qwen.py:704: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [8]:
processed_data[1]

{'values': array([-1.039665  ,  0.15772526,  1.5772394 , ..., -0.14516646,
        -0.13970429,  3.0700932 ], dtype=float32),
 'id': 'Prof. Michael Chen',
 'metadata': {'review': 'Brilliant instructor! Prof. Chen makes complex algorithms easy to understand. His projects are challenging but rewarding.',
  'subjects': 'Computer Science',
  'stars': 5}}

In [9]:
index = pc.Index('rag')
index.upsert(
    vectors=processed_data,
    namespace='ns1'
)

{'upserted_count': 20}

In [11]:
index.describe_index_stats()

{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}